In [1]:
import pandas as pd
import numpy as np
import datetime
import re
from datetime import timedelta
from datetime import date

## Carga de DATAFRAMES

In [2]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/13. Onstar/Onstar.xlsx"
nombre_hoja = 'Onstar'
df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

In [3]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/24.VS Code/AgendaLarga/UltimoEstadoAgendaLargaALL.xlsx"
nombre_hoja = 'Sheet1'
df_agenda = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

In [4]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/24.VS Code/Facturacion/UltimoEstadoSSTT.xlsx"
nombre_hoja = 'Sheet1'
df_sstt = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

In [5]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/24.VS Code/Libromatico/UltimoRegistroLibrmaticoALL.xlsx"
nombre_hoja = 'Sheet1'
df_libromatico = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

## Normalizacion de la data

In [6]:
nuevos_nombres = ["make","model_year","model","vin","patente_onstar","last_report","customer","errors","warnings","odmeter","total_fuel_usage","avg_fuel_usage","tags","odometro","rango"]
df.columns = nuevos_nombres

In [7]:
df = df.astype(str)
df["last_report"] = pd.to_datetime(df["last_report"], format="%Y-%m-%d")

## Union Onstar con AgendaALL

In [8]:
df_merge = pd.merge(pd.merge(pd.merge(df, df_agenda, on='vin', how='left'), df_sstt, on='vin', how='left'), df_libromatico, on='vin', how='left')


In [9]:
df_merge = df_merge[['vin', 'patente_onstar', 'model', 'model_year','last_report', 'odometro','rango',
                     'fecha_reserva', 'trabajo_agenda', 'patente_x', 'trabajo', 'estado', 'vigencia','ciclo',
                      'oficina_y','patente_y','tipo_trabajo', 'fecha_venta_x', 'ultima_mantencion_y', 'ultima_fecha_y',
                      'oficina','patente_consolidado', 'modelo_y','categoria','rut_cliente','cliente_2', 'movil', 'fono', 'fono_2', 'email']]


In [10]:
df_merge['patente_onstar'] = df_merge['patente_onstar'].replace('[^a-zA-Z0-9]', '', regex=True)  # Eliminar caracteres no deseados
df_merge['patente_onstar'] = df_merge['patente_onstar'].str.strip()  # Eliminar espacios en blanco
df_merge['patente_onstar'] = df_merge['patente_onstar'].str.upper()  # Convertir a mayúsculas


In [11]:
conditions = [    df_merge['patente_consolidado'].notnull() & (df_merge['patente_consolidado'] != 'nan') & (df_merge['patente_consolidado'] != 'None') & (df_merge['patente_consolidado'] != 'Registra Cliente'),  # si patente_consolidado no es None ni NaN ni una cadena vacía ni 'Registra Cliente'
    df_merge['patente_onstar'].notnull() & (df_merge['patente_onstar'] != 'NAN') & (df_merge['patente_onstar'] != '.'),           # si patente_onstar no es None ni NaN ni una cadena vacía
    df_merge['patente_x'].notnull() & (df_merge['patente_x'] != 'nan') & (df_merge['patente_x'] != 'None'),                     # si patente_x no es None ni NaN ni una cadena vacía
    df_merge['patente_y'].notnull() & (df_merge['patente_y'] != 'nan') & (df_merge['patente_y'] != 'None'),                     # si patente_y no es None ni NaN ni una cadena vacía
]
choices = [    df_merge['patente_consolidado'],    # asignar los valores de patente_consolidado
    df_merge['patente_onstar'],         # asignar los valores de patente_onstar
    df_merge['patente_x'],              # asignar los valores de patente_x
    df_merge['patente_y'],              # asignar los valores de patente_y
]

df_merge.loc[:, 'patente_all'] = np.select(conditions, choices, default=None)


In [12]:
# Aquí se reemplazan los valores faltantes y infinitos en la columna 'movil'
df_merge['movil'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_merge['movil'].fillna(value=0, inplace=True)

# Aquí se convierte la columna 'movil' de flotantes a enteros y se elimina el ".0"
df_merge['movil'] = df_merge['movil'].astype(int)

# Aquí se imprime el dataframe modificado
#print(df_merge)


In [13]:
# Aquí se reemplazan los valores faltantes y infinitos en las columnas 'fono', 'fono_2', 'ultima_mantencion_y' y 'movil'
df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']] = df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']].replace([np.inf, -np.inf], np.nan)
df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']] = df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']].fillna(value=0)

# Aquí se convierten las columnas 'fono', 'fono_2', 'ultima_mantencion_y' y 'movil' de flotantes a enteros y se eliminan los ".0"
df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']] = df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']].astype(int)

# Aquí se imprime el dataframe modificado
#print(df_merge)


In [14]:
df_merge = df_merge.astype(str)


# Aquí se reemplazan los valores faltantes y infinitos en las columnas 'fono', 'fono_2', 'ultima_mantencion_y' y 'movil'
df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']] = df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']].replace([np.inf, -np.inf], np.nan)
df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']] = df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']].fillna(value=0)

# Aquí se convierten las columnas 'fono', 'fono_2', 'ultima_mantencion_y' y 'movil' de flotantes a enteros y se eliminan los ".0"
df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']] = df_merge[['fono', 'fono_2', 'ultima_mantencion_y', 'movil']].astype(int)

df_merge["last_report"] = pd.to_datetime(df_merge["last_report"], format="%Y-%m-%d")
df_merge["fecha_reserva"] = pd.to_datetime(df_merge["fecha_reserva"], format="%Y-%m-%d")
df_merge["fecha_venta_x"] = pd.to_datetime(df_merge["fecha_venta_x"], format="%Y-%m-%d")
df_merge["ultima_fecha_y"] = pd.to_datetime(df_merge["ultima_fecha_y"], format="%Y-%m-%d")


df_merge.to_excel("OnstarAgenda.xlsx", index=False)